In [1]:
from google.colab import drive
drive.mount('/content/drive')

!ls
!ls /content/drive/MyDrive/nlp_project

!pip install datasets
!pip install transformers
!pip install evaluate

import os
os.environ["WANDB_DISABLED"] = "true"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data
1raw_lyrics_test_dataset.csv   raw_lyrics_test_dataset.gsheet
1raw_lyrics_train_dataset.csv  raw_lyrics_train_dataset.csv
cleaned_data.csv	       raw_lyrics_train_dataset.gsheet
raw_lyrics_test_dataset.csv    results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account a

In [2]:
from datasets import load_dataset
clickbait = load_dataset('csv',data_files={'train': '/content/drive/MyDrive/nlp_project/cleaned_data.csv',
                                           'test': '/content/drive/MyDrive/nlp_project/raw_lyrics_test_dataset.csv'})



Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [57]:
small_train_dataset = clickbait["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = clickbait["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [58]:
print("Lyrics:", small_train_dataset[0]["Lyrics"])
print("Genre_Label:", small_train_dataset[0]["label"])

Lyrics: Lyrics[Verse 1] Well, when you go Don't ever think I'll make you try to stay And maybe when you get back I'll be off to find another way  [Pre-Chorus] And after all this time that you still owe You're still a good-for-nothing, I don't know So take your gloves and get out Better get out while you can  [Chorus] When you go, and would you even turn to say "I don't love you like I did yesterday"?  [Verse 2] Sometimes I cry so hard from pleading So sick and tired of all the needless beating But baby, when they knock you down and out Is where you oughta stay  [Pre-Chorus] And after all the blood that you still owe Another dollar's just another blow So fix your eyes and get up Better get up while you can, whoa, whoa, whoa   [Chorus] When you go, and would you even turn to say "I don't love you like I did yesterday"?  [Post-Chorus] Well, come on, come on [Guitar Solo]  [Bridge] When you go, would you have the guts to say "I don't love you like I loved you yesterday"?  [Outro] I don't l

In [59]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [60]:
def preprocess_function(examples):
    return tokenizer(examples["Lyrics"], truncation=True)


In [61]:
# (removing batched=True)
tokenized_small_train = small_train_dataset.map(preprocess_function)
tokenized_small_test = small_test_dataset.map(preprocess_function)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [63]:
# Let's look at the first training sentence

print(tokenized_small_train)

print("Lyrics:", tokenized_small_train[0]["Lyrics"])
print("Genre_Label:", tokenized_small_train[0]["label"])
print("Input IDs:", tokenized_small_train[0]["input_ids"])
print("Attention Mask:", tokenized_small_train[0]["attention_mask"])

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(tokenized_small_train[0]["input_ids"])
print("Tokenized text:", tokens)

Dataset({
    features: ['Lyrics', 'label', 'input_ids', 'attention_mask'],
    num_rows: 3000
})
Lyrics: Lyrics[Verse 1] Well, when you go Don't ever think I'll make you try to stay And maybe when you get back I'll be off to find another way  [Pre-Chorus] And after all this time that you still owe You're still a good-for-nothing, I don't know So take your gloves and get out Better get out while you can  [Chorus] When you go, and would you even turn to say "I don't love you like I did yesterday"?  [Verse 2] Sometimes I cry so hard from pleading So sick and tired of all the needless beating But baby, when they knock you down and out Is where you oughta stay  [Pre-Chorus] And after all the blood that you still owe Another dollar's just another blow So fix your eyes and get up Better get up while you can, whoa, whoa, whoa   [Chorus] When you go, and would you even turn to say "I don't love you like I did yesterday"?  [Post-Chorus] Well, come on, come on [Guitar Solo]  [Bridge] When you go

In [64]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [65]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [79]:
import numpy as np
from evaluate import load


def compute_metrics(eval_pred):
   load_accuracy = load("accuracy")
   load_f1 = load("f1")
   load_recall = load("recall")
   load_precision = load("precision")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
   recall = load_recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
   precision = load_precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
   return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

In [80]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/nlp_project/results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_small_train,
    eval_dataset=tokenized_small_test,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [81]:

print(tokenized_small_train[0])


{'Lyrics': 'Lyrics[Verse 1] Well, when you go Don\'t ever think I\'ll make you try to stay And maybe when you get back I\'ll be off to find another way  [Pre-Chorus] And after all this time that you still owe You\'re still a good-for-nothing, I don\'t know So take your gloves and get out Better get out while you can  [Chorus] When you go, and would you even turn to say "I don\'t love you like I did yesterday"?  [Verse 2] Sometimes I cry so hard from pleading So sick and tired of all the needless beating But baby, when they knock you down and out Is where you oughta stay  [Pre-Chorus] And after all the blood that you still owe Another dollar\'s just another blow So fix your eyes and get up Better get up while you can, whoa, whoa, whoa   [Chorus] When you go, and would you even turn to say "I don\'t love you like I did yesterday"?  [Post-Chorus] Well, come on, come on [Guitar Solo]  [Bridge] When you go, would you have the guts to say "I don\'t love you like I loved you yesterday"?  [Out

In [82]:
trainer.train()

Step,Training Loss
500,0.917800


TrainOutput(global_step=940, training_loss=0.7255181008196891, metrics={'train_runtime': 703.9776, 'train_samples_per_second': 21.307, 'train_steps_per_second': 1.335, 'total_flos': 1987152721920000.0, 'train_loss': 0.7255181008196891, 'epoch': 5.0})

In [83]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

{'eval_loss': 1.0188297033309937, 'eval_accuracy': 0.6466666666666666, 'eval_f1': 0.6423506031130395, 'eval_recall': 0.6466666666666666, 'eval_precision': 0.6433570351403975, 'eval_runtime': 10.052, 'eval_samples_per_second': 29.845, 'eval_steps_per_second': 1.89, 'epoch': 5.0}
